Table 2: Alcohol-specific age-specific death rates per 100,000 population, deaths registered in the United Kingdom, 2001 to 2016

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/datasets/alcoholspecificdeathsintheukmaindataset')
    sheet = scraper.distribution().as_databaker()

In [2]:
tab = next(tab for tab in sheet if tab.name=='Table 2')

In [3]:
observations = tab.excel_ref('E20').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
observations

{<G252 'z'>, <K228 18.0>, <H310 0.5>, <N328 'z'>, <F74 10.5>, <L142 6.1>, <S136 'z'>, <R320 28.0>, <R142 11.9>, <M200 8.9>, <L289 'z'>, <S60 'z'>, <R81 0.2>, <G126 27.6>, <G287 'z'>, <K173 0.0>, <K144 321.0>, <M296 13.2>, <N333 12.1>, <N142 7.1>, <N232 'z'>, <S34 24.5>, <F237 8.2>, <N200 11.5>, <S290 'z'>, <Q330 47.0>, <M87 13.2>, <N285 2.9>, <T300 38.0>, <O171 'u'>, <T202 39.8>, <E150 177.0>, <E302 244.0>, <N282 11.2>, <Q88 671.0>, <S140 1.7>, <L276 9.6>, <L203 19.7>, <R52 35.6>, <T242 39.0>, <F108 28.2>, <L102 1.0>, <T320 30.8>, <O304 'u'>, <F240 23.1>, <F190 3.5>, <E223 1055.0>, <H62 0.3>, <H224 23.9>, <T41 'z'>, <F214 'z'>, <L67 9.2>, <M204 14.5>, <F238 14.4>, <S322 11.6>, <F146 30.2>, <H222 31.8>, <F286 'z'>, <M110 11.2>, <M304 0.6>, <N89 18.2>, <K92 120.0>, <F60 'z'>, <H155 'z'>, <S239 24.9>, <G317 26.0>, <F40 'z'>, <N254 1.2>, <Q157 5.0>, <Q108 742.0>, <L326 'z'>, <S208 6.4>, <L266 0.9>, <M196 0.0>, <N167 17.4>, <S48 18.6>, <H313 8.1>, <T307 'z'>, <S246 9.8>, <Q161 310.0>, <F177

In [5]:
Sex = tab.excel_ref('E18').expand(RIGHT).is_not_whitespace()
Sex

{<E18 'All persons'>, <Q18 'Males'>, <K18 'Females'>}

In [6]:
deaths = tab.excel_ref('E19').expand(RIGHT).is_not_blank()
deaths

{<Q19 'Deaths'>, <R19 'Rate per 100,000 females1'>, <M19 'Lower 95% confidence limit'>, <S19 'Lower 95% confidence limit'>, <E19 'Deaths'>, <L19 'Rate per 100,000 males1'>, <T19 'Upper 95% confidence limit'>, <K19 'Deaths'>, <F19 'Rate per 100,000 people1'>, <G19 'Lower 95% confidence limit'>, <H19 'Upper 95% confidence limit'>, <N19 'Upper 95% confidence limit'>}

In [7]:
Year = tab.excel_ref('B20').expand(DOWN).is_not_blank() - tab.excel_ref('A343').expand(DOWN)  
Year

{<B39 2002.0>, <B210 2011.0>, <B58 2003.0>, <B191 2010.0>, <B172 2009.0>, <B134 2007.0>, <B324 2017.0>, <B267 2014.0>, <B248 2013.0>, <B153 2008.0>, <B305 2016.0>, <B115 2006.0>, <B96 2005.0>, <B286 2015.0>, <B77 2004.0>, <B20 2001.0>, <B229 2012.0>}

In [8]:
Age = tab.excel_ref('C20').expand(DOWN).is_not_blank() - tab.excel_ref('B343').expand(DOWN)  
Age

{<C97 '01-04'>, <C314 '40-44'>, <C23 '10-14'>, <C196 '20-24'>, <C139 '20-24'>, <C278 '50-54'>, <C149 '70-74'>, <C280 '60-64'>, <C141 '30-34'>, <C135 '01-04'>, <C216 '25-29'>, <C199 '35-39'>, <C29 '40-44'>, <C320 '70-74'>, <C92 '70-74'>, <C253 '20-24'>, <C322 '80-84'>, <C138 '15-19'>, <C197 '25-29'>, <C325 '01-04'>, <C298 '55-59'>, <C285 '85+'>, <C134 '<1'>, <C116 '01-04'>, <C338 '65-69'>, <C312 '30-34'>, <C286 '<1'>, <C155 '05-09'>, <C324 '<1'>, <C77 '<1'>, <C183 '50-54'>, <C323 '85+'>, <C107 '50-54'>, <C132 '80-84'>, <C260 '55-59'>, <C178 '25-29'>, <C306 '01-04'>, <C57 '85+'>, <C303 '80-84'>, <C123 '35-39'>, <C305 '<1'>, <C55 '75-79'>, <C167 '65-69'>, <C182 '45-49'>, <C237 '35-39'>, <C270 '10-14'>, <C258 '45-49'>, <C200 '40-44'>, <C65 '30-34'>, <C224 '65-69'>, <C226 '75-79'>, <C41 '05-09'>, <C46 '30-34'>, <C180 '35-39'>, <C101 '20-24'>, <C190 '85+'>, <C176 '15-19'>, <C110 '65-69'>, <C179 '30-34'>, <C160 '30-34'>, <C251 '10-14'>, <C66 '35-39'>, <C49 '45-49'>, <C184 '55-59'>, <C310 '20-

In [9]:
Dimensions = [
            HDim(Year,'Year',CLOSEST,ABOVE),
            HDim(Age,'Age',DIRECTLY,LEFT),
            HDim(Sex,'Sex',CLOSEST,LEFT),
            HDim(deaths, 'Alcohol Specific Deaths',DIRECTLY,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')
            ]

In [10]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [11]:
new_table = c1.topandas()
new_table

,OBS,DATAMARKER,Year,Age,Sex,Alcohol Specific Deaths,Measure Type,Unit
0,0,NaN,2001.0,<1,All persons,Deaths,Count,People
1,,z,2001.0,<1,All persons,"Rate per 100,000 people1",Count,People
2,,z,2001.0,<1,All persons,Lower 95% confidence limit,Count,People
3,,z,2001.0,<1,All persons,Upper 95% confidence limit,Count,People
4,0,NaN,2001.0,<1,Females,Deaths,Count,People
5,,z,2001.0,<1,Females,"Rate per 100,000 males1",Count,People
6,,z,2001.0,<1,Females,Lower 95% confidence limit,Count,People
7,,z,2001.0,<1,Females,Upper 95% confidence limit,Count,People
8,0,NaN,2001.0,<1,Males,Deaths,Count,People
9,,z,2001.0,<1,Males,"Rate per 100,000 females1",Count,People


In [12]:
new_table = new_table[new_table['OBS'] !=  0 ]

In [13]:
new_table = new_table[new_table['OBS'] !=  'z' ]

In [14]:
new_table = new_table[new_table['OBS'] !=  '' ]

In [15]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [16]:
new_table['Year'] = new_table['Year'].astype(int)

In [17]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [18]:
new_table.dtypes

Value                      object
DATAMARKER                 object
Year                        int32
Age                        object
Sex                        object
Alcohol Specific Deaths    object
Measure Type               object
Unit                       object
dtype: object

In [19]:
new_table

,Value,DATAMARKER,Year,Age,Sex,Alcohol Specific Deaths,Measure Type,Unit
36,1,NaN,2001,10-14,All persons,Deaths,Count,People
40,1,NaN,2001,10-14,Females,Deaths,Count,People
48,11,NaN,2001,15-19,All persons,Deaths,Count,People
49,0.3,NaN,2001,15-19,All persons,"Rate per 100,000 people1",Count,People
50,0.1,NaN,2001,15-19,All persons,Lower 95% confidence limit,Count,People
51,0.5,NaN,2001,15-19,All persons,Upper 95% confidence limit,Count,People
53,1,NaN,2001,15-19,Females,Deaths,Count,People
57,10,NaN,2001,15-19,Males,Deaths,Count,People
58,0.5,NaN,2001,15-19,Males,"Rate per 100,000 females1",Count,People
59,0.3,NaN,2001,15-19,Males,Lower 95% confidence limit,Count,People


In [20]:
nt1 = new_table[new_table['Alcohol Specific Deaths'] == 'Lower 95% confidence limit']

In [21]:
nt1

,Value,DATAMARKER,Year,Age,Sex,Alcohol Specific Deaths,Measure Type,Unit
50,0.1,NaN,2001,15-19,All persons,Lower 95% confidence limit,Count,People
59,0.3,NaN,2001,15-19,Males,Lower 95% confidence limit,Count,People
64,0.3,NaN,2001,20-24,All persons,Lower 95% confidence limit,Count,People
73,0.5,NaN,2001,20-24,Males,Lower 95% confidence limit,Count,People
78,1.2,NaN,2001,25-29,All persons,Lower 95% confidence limit,Count,People
82,0.5,NaN,2001,25-29,Females,Lower 95% confidence limit,Count,People
87,1.6,NaN,2001,25-29,Males,Lower 95% confidence limit,Count,People
91,3.6,NaN,2001,30-34,All persons,Lower 95% confidence limit,Count,People
95,1.8,NaN,2001,30-34,Females,Lower 95% confidence limit,Count,People
99,5,NaN,2001,30-34,Males,Lower 95% confidence limit,Count,People


In [22]:
nt1.columns = ['Lower 95% confidence limit' if x=='Value' else x for x in new_table.columns]

In [23]:
nt2 = new_table[new_table['Alcohol Specific Deaths'] == 'Upper 95% confidence limit']

In [24]:
nt2

,Value,DATAMARKER,Year,Age,Sex,Alcohol Specific Deaths,Measure Type,Unit
51,0.5,NaN,2001,15-19,All persons,Upper 95% confidence limit,Count,People
60,1,NaN,2001,15-19,Males,Upper 95% confidence limit,Count,People
65,0.7,NaN,2001,20-24,All persons,Upper 95% confidence limit,Count,People
74,1.4,NaN,2001,20-24,Males,Upper 95% confidence limit,Count,People
79,2,NaN,2001,25-29,All persons,Upper 95% confidence limit,Count,People
83,1.3,NaN,2001,25-29,Females,Upper 95% confidence limit,Count,People
88,3,NaN,2001,25-29,Males,Upper 95% confidence limit,Count,People
92,4.7,NaN,2001,30-34,All persons,Upper 95% confidence limit,Count,People
96,3.1,NaN,2001,30-34,Females,Upper 95% confidence limit,Count,People
100,7,NaN,2001,30-34,Males,Upper 95% confidence limit,Count,People


In [25]:
nt2.columns = ['Upper 95% confidence limit' if x=='Value' else x for x in new_table.columns]

In [26]:
new_table.count()

Value                      2940
DATAMARKER                    0
Year                       2940
Age                        2940
Sex                        2940
Alcohol Specific Deaths    2940
Measure Type               2940
Unit                       2940
dtype: int64

In [27]:
new_table = new_table[new_table['Alcohol Specific Deaths'] != 'Upper 95% confidence limit']

In [28]:
new_table = new_table[new_table['Alcohol Specific Deaths'] != 'Lower 95% confidence limit']

In [29]:
Final_table = pd.merge(new_table, nt1, how = 'outer', on = ['Year','Sex','Age'])

In [30]:
Final_table.head(2)

,Value,DATAMARKER_x,Year,Age,Sex,Alcohol Specific Deaths_x,Measure Type_x,Unit_x,Lower 95% confidence limit,DATAMARKER_y,Alcohol Specific Deaths_y,Measure Type_y,Unit_y
0,1,NaN,2001,10-14,All persons,Deaths,Count,People,NaN,NaN,NaN,NaN,NaN
1,1,NaN,2001,10-14,Females,Deaths,Count,People,NaN,NaN,NaN,NaN,NaN


In [31]:
Final_table = Final_table[['Value','Year','Age','Sex','Alcohol Specific Deaths_x','Measure Type_x','Unit_x',
                           'Lower 95% confidence limit']]

In [32]:
Final_table = pd.merge(Final_table, nt2, how = 'outer', on = ['Year','Sex','Age'])

In [33]:
Final_table.head(1)

,Value,Year,Age,Sex,Alcohol Specific Deaths_x,Measure Type_x,Unit_x,Lower 95% confidence limit,Upper 95% confidence limit,DATAMARKER,Alcohol Specific Deaths,Measure Type,Unit
0,1,2001,10-14,All persons,Deaths,Count,People,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
Final_table = Final_table[['Year','Sex','Age','Alcohol Specific Deaths_x','Measure Type','Value','Lower 95% confidence limit', 'Upper 95% confidence limit','Unit']]

In [35]:
Final_table

,Year,Sex,Age,Alcohol Specific Deaths_x,Measure Type,Value,Lower 95% confidence limit,Upper 95% confidence limit,Unit
0,2001,All persons,10-14,Deaths,NaN,1,NaN,NaN,NaN
1,2001,Females,10-14,Deaths,NaN,1,NaN,NaN,NaN
2,2001,All persons,15-19,Deaths,Count,11,0.1,0.5,People
3,2001,All persons,15-19,"Rate per 100,000 people1",Count,0.3,0.1,0.5,People
4,2001,Females,15-19,Deaths,NaN,1,NaN,NaN,NaN
5,2001,Males,15-19,Deaths,Count,10,0.3,1,People
6,2001,Males,15-19,"Rate per 100,000 females1",Count,0.5,0.3,1,People
7,2001,All persons,20-24,Deaths,Count,16,0.3,0.7,People
8,2001,All persons,20-24,"Rate per 100,000 people1",Count,0.4,0.3,0.7,People
9,2001,Females,20-24,Deaths,NaN,1,NaN,NaN,NaN


In [36]:
Final_table.columns = ['Alcohol Specific Deaths' if x=='Alcohol Specific Deaths_x' else x for x in Final_table.columns]

In [37]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return ''
    else:
        return y
    
Final_table['Lower 95% confidence limit'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Lower 95% confidence limit']), axis = 1)

In [38]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return ''
    else:
        return y
    
Final_table['Upper 95% confidence limit'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Upper 95% confidence limit']), axis = 1)

In [39]:
Final_table['Alcohol Specific Deaths'] = Final_table['Alcohol Specific Deaths'].str.rstrip('1')

In [40]:
Final_table['Sex'] = Final_table['Sex'].map(
    lambda x: {
        'All persons' : 'T', 
        'Males' : 'M',
        'Females': 'F'         
        }.get(x, x))

In [41]:
Final_table['Geography'] =  'K02000001'

In [42]:
Final_table

,Year,Sex,Age,Alcohol Specific Deaths,Measure Type,Value,Lower 95% confidence limit,Upper 95% confidence limit,Unit,Geography
0,2001,T,10-14,Deaths,NaN,1,,,NaN,K02000001
1,2001,F,10-14,Deaths,NaN,1,,,NaN,K02000001
2,2001,T,15-19,Deaths,Count,11,,,People,K02000001
3,2001,T,15-19,"Rate per 100,000 people",Count,0.3,0.1,0.5,People,K02000001
4,2001,F,15-19,Deaths,NaN,1,,,NaN,K02000001
5,2001,M,15-19,Deaths,Count,10,,,People,K02000001
6,2001,M,15-19,"Rate per 100,000 females",Count,0.5,0.3,1,People,K02000001
7,2001,T,20-24,Deaths,Count,16,,,People,K02000001
8,2001,T,20-24,"Rate per 100,000 people",Count,0.4,0.3,0.7,People,K02000001
9,2001,F,20-24,Deaths,NaN,1,,,NaN,K02000001


In [44]:
Final_table['Alcohol Specific Deaths'].unique()

array(['Deaths', 'Rate per 100,000 people', 'Rate per 100,000 females',
       'Rate per 100,000 males'], dtype=object)

In [ ]:
Final_table['Alcohol Specific Deaths'] = Final_table['Alcohol Specific Deaths'].map(
    lambda x: {
        'Rate per 100,000 males' : 'Rate per 100,000 females', 
        'Rate per 100,000 females' : 'Rate per 100,000 males',
        }.get(x, x))